In [ ]:
import pandas as pd
import os
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold

from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import mean_squared_error

#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [ ]:
# List all datasets and accompanying files
DATA_PATH = "Users\梁浩弘\OneDrive\学习\UniMelb\Year2\Semester2\Eodp\Ass2\a2"

for root, dirs, files in os.walk(DATA_PATH):
        level = root.replace(DATA_PATH, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

Data cleaning 

In [ ]:
# open the dataset of each region and convert into dataframe 
f = open(DATA_PATH+'/games/EUmatch.csv', "r")
EU = pd.read_csv(f)
EU = pd.DataFrame(EU)

f = open(DATA_PATH+'/games/KRmatch.csv', "r")
KR = pd.read_csv(f)
KR = pd.DataFrame(KR)

f = open(DATA_PATH+'/games/NAmatch.csv', "r")
NA = pd.read_csv(f)
NA = pd.DataFrame(NA)

# step 1： concat the 3 dataset into 1 dataframe
info = pd.concat([EU, KR, NA])

In [ ]:
# step 2: drop the nah value in dataframe
info = info.dropna()

In [ ]:
# step 3: using convert type function to change string into unique int 
info=info.copy()
# change the category of the data into digital
def convert_type(data,col_list):
    columns = col_list
    data[columns] = data[columns].astype("category")
    for col in columns:
        data[col]=data[col].cat.codes
    return data

info = convert_type(info,["champion","side","role","minions_killed"])

In [ ]:
# step 4: remove the outlier by using the quantile 

from matplotlib.lines import lineStyles


info = info.sort_values("kda")
# draw boxplot of kda
info["kda"].plot.box(title="Boxplot of KDA Before Removing Outliers")
plt.show()
# calculate quartiles
kdaAve = info['kda'].mean()
kdaQ1 = info['kda'].quantile(q=0.25)
kdaQ3 = info['kda'].quantile(q=0.75)
kdaIQR = kdaQ3 - kdaQ1

# show upper outliers and lower outliers
print("upper outlier is %f", kdaQ3 + 3 * kdaIQR)
print("lower outlier is %f", kdaQ1 - 3 * kdaIQR)

# drop outliers
info_remove_out = info.drop(info[(info['kda'] < (kdaQ1 - 3 * kdaIQR)) | (info['kda'] > (kdaQ3 + 3 * kdaIQR))].index)

In [ ]:

level = info_remove_out['level']
plt.hist(level)

In [ ]:

info_remove_out.drop(info_remove_out [info_remove_out['level'] < 10].index, inplace = True)

print(info_remove_out)


Preliminary Analysis (here contain the feature selection)

In [ ]:
# Heatmap before remove outlier 
plt.figure(figsize = (16,9))
corr = info.corr()
sns.heatmap(corr, cmap= 'coolwarm', annot = True)
plt.show()

In [ ]:
# Heatmap after remove outlier 
plt.figure(figsize = (16,9))
corr = info_remove_out.corr()
sns.heatmap(corr, cmap= 'coolwarm', annot = True)
plt.show()

In [ ]:
# clean the dataset using background information 
# (like we remove kill, death, assist as they are directly depended on kda)
df = info_remove_out
df = df.drop(["kills", "deaths", "assists"], axis = 1)

In [ ]:
# Drop the corraltion which is lower than 0.05
corrlation = df.corr(method='pearson')['kda'].sort_values()
abs_corr = abs(corrlation).sort_values()
print(abs_corr)

In [ ]:
# remove the less correlated features 
df = df.drop([ "role", "d_spell", "f_spell", "side"], axis = 1)

Train Test set split

In [ ]:
# spilt into training set and test set
X=df.drop(columns='kda')
Y=df['kda']
X_train, X_test, Y_train, Y_test= train_test_split(X,Y,test_size=0.2, random_state = 0)

plt.hist(Y,bins=100)
plt.xlabel("KDA score")
plt.ylabel("frequency")
plt.show()

In [ ]:
#Backward Elimination (for feature selection)
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(Y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))
# draw scatter plot 

ax.scatter(X_train['damage_objectives'], Y_train, color='Purple')
ax.set_xlabel('damage_objective')
ax.set_ylabel('TOTALDEMAND')

z = np.polyfit(X_train['damage_objectives'], Y_train, 1)
p = np.poly1d(z)
#add trendline to plot
plt.plot(X_train['damage_objectives'], p(X_train['damage_objectives']), color = 'red')

plt.show()




Model

In [ ]:
# linear regression model
reg = LinearRegression().fit(X_train,Y_train)
cv = RepeatedKFold(n_splits = 10,n_repeats=3, random_state=0)
r_sq_1 = reg.score(X_train,Y_train)
r_sq_2 = reg.score(X_test,Y_test)

print('train', r_sq_1)
print('test', r_sq_2)
#print(cross_val_score(reg, X_train, Y_train, cv=5).mean())

# linear regression model MSE
Y_pred = reg.predict(X_test)
mse = mean_squared_error(Y_test,Y_pred)
rmse = mse**.5
print('mse', mse)
print('rmse', rmse)

In [ ]:
# Decision Tree regression
from sklearn import tree
import matplotlib.pyplot as plt 
treebase = DecisionTreeRegressor(random_state=0)
treebase.fit(X_train, Y_train)
print("train ", treebase.score(X_train, Y_train))
print("test", treebase.score(X_test, Y_test))
print(cross_val_score(treebase, X_train, Y_train, cv=5).mean())


# regression tree
clf = tree.DecisionTreeRegressor(max_depth=3)
clf.fit(X_train, Y_train)
print("test",clf.score(X_test, Y_test))
plt.figure(figsize=(10,10))
tree.plot_tree(clf)
plt.show()

In [ ]:
# Random Forest regression
RF = RandomForestRegressor(n_estimators=100)
RF.fit(X_train, Y_train)
print("train ", RF.score(X_train, Y_train))
print("test",RF.score(X_test, Y_test))
print(cross_val_score(RF, X_train, Y_train, cv=5).mean())



In [ ]:
# feature importance 
feature_names = [f"{name}" for name in X_train.head()]
importances = RF.feature_importances_
std = np.std([tree.feature_importances_ for tree in RF.estimators_], axis=0)

In [ ]:
# plot the feature importance graph and values
forest_importances = pd.Series(importances, index=feature_names)
print(forest_importances)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()